https://vk.com/papersreaders?w=wall-154085965_8

In [20]:
from sklearn.metrics import r2_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from torch import optim
import random
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn

In [21]:
criteo = pd.read_csv("https://raw.githubusercontent.com/shenweichen/DeepCTR-Torch/master/examples/criteo_sample.txt")

In [22]:
sparse_fields = criteo.columns[criteo.columns.str.startswith("C")].tolist()
dense_fields = criteo.columns[criteo.columns.str.startswith("I")].tolist()

In [23]:
criteo = criteo[sparse_fields]

In [24]:
criteo.fillna({f: "missing" for f in sparse_fields}, inplace=True)
criteo.fillna({f: 0 for f in dense_fields}, inplace=True)

In [35]:
X = criteo.drop(columns=["label"], errors="ignore").copy()

In [36]:
criteo_dummy = pd.get_dummies(X, columns=sparse_fields, prefix_sep="_")
criteo_dummy.shape

(200, 2278)

In [26]:
criteo_dummy.head()

,C1_05db9164,C1_09ca0b81,C1_0e78bd46,C1_17f69355,C1_241546e0,C1_2d4ea12b,C1_39af2607,C1_3b65d647,C1_52f1e825,C1_5a9ed9b0,...,C26_e438a496,C26_e75c9ae9,C26_f3737bd0,C26_f3b1f00d,C26_f4642e0e,C26_f610730e,C26_f89dfbcc,C26_f95af538,C26_fa3124de,C26_missing
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
features_by_field = []

# for field in dense_fields:
#     features_by_field.append({
#         "field_name": field,
#         "type": "dense",
#         "feature_names": [field],
#         "feature_indeces": [criteo_dummy.columns.get_loc(field)]
#     })
    
for field in sparse_fields:
    features = sorted(X[field].unique().tolist())
    
    features_by_field.append({
        "field_name": field,
        "type": "sparse",
        "feature_names": features,
        "feature_indeces": [criteo_dummy.columns.get_loc(f"{field}_{f}") for f in features]
    })
len(features_by_field)

26

In [51]:
field = list(filter(lambda x: x["type"] == "sparse", features_by_field))[0]

Каждый признак (разложенный до уровня категории) умножаем на каждое поле (кроме его собственного).

In [81]:
weights1 = set()

for c in criteo_dummy:
    current_field = c.split("_")[0]
    feature = c.split("_")[1]
    for f in criteo:
        if current_field != f and ((f, current_field, feature) not in weights1):
            weights1.add((current_field, f, feature))
len(weights1)

56883

In [79]:
weights2 = set()
for field1 in X:
    for field2 in X:
        if field1 != field2:
            for category in X[field1].unique():
                if (field2, field1, category) not in weights2:
                    weights2.add((field1, field2, category))
len(weights2)

56883

In [67]:
len(weights2 - weights1)

914

In [70]:
criteo_dummy["C1_05db9164"]

0      1
1      0
2      1
3      1
4      1
5      0
6      1
7      1
8      0
9      1
10     1
11     1
12     1
13     0
14     0
15     1
16     0
17     1
18     0
19     0
20     0
21     1
22     0
23     0
24     0
25     0
26     1
27     1
28     0
29     0
      ..
170    0
171    0
172    1
173    0
174    1
175    1
176    1
177    0
178    0
179    1
180    0
181    0
182    0
183    1
184    0
185    0
186    1
187    0
188    0
189    0
190    0
191    1
192    0
193    0
194    1
195    1
196    0
197    1
198    1
199    0
Name: C1_05db9164, Length: 200, dtype: uint8

In [75]:
"C1_05db9164" in weights1

False

In [73]:
"05db9164" in X["C1"].unique()

True

In [68]:
weights2 - weights1

{('C1', 'C10', '05db9164'),
 ('C1', 'C10', '09ca0b81'),
 ('C1', 'C10', '0e78bd46'),
 ('C1', 'C10', '17f69355'),
 ('C1', 'C10', '241546e0'),
 ('C1', 'C10', '2d4ea12b'),
 ('C1', 'C10', '39af2607'),
 ('C1', 'C10', '3b65d647'),
 ('C1', 'C10', '52f1e825'),
 ('C1', 'C10', '5a9ed9b0'),
 ('C1', 'C10', '5bfa8ab5'),
 ('C1', 'C10', '68fd1e64'),
 ('C1', 'C10', '75ac2fe6'),
 ('C1', 'C10', '7e5c2ff4'),
 ('C1', 'C10', '87552397'),
 ('C1', 'C10', '87773c45'),
 ('C1', 'C10', '8cf07265'),
 ('C1', 'C10', '98237733'),
 ('C1', 'C10', '9a89b36c'),
 ('C1', 'C10', 'ae82ea21'),
 ('C1', 'C10', 'be30ca83'),
 ('C1', 'C10', 'be589b51'),
 ('C1', 'C10', 'da4eff0f'),
 ('C1', 'C10', 'de4dac42'),
 ('C1', 'C10', 'f434fac1'),
 ('C1', 'C10', 'f473b8dc'),
 ('C1', 'C10', 'fc9c62bb'),
 ('C1', 'C11', '05db9164'),
 ('C1', 'C11', '09ca0b81'),
 ('C1', 'C11', '0e78bd46'),
 ('C1', 'C11', '17f69355'),
 ('C1', 'C11', '241546e0'),
 ('C1', 'C11', '2d4ea12b'),
 ('C1', 'C11', '39af2607'),
 ('C1', 'C11', '3b65d647'),
 ('C1', 'C11', '52f1

In [53]:
class FM(nn.Module):
    def __init__(self, features_num=None, k=2):
        super().__init__()
        self.V = nn.Parameter(torch.randn(features_num, k), requires_grad=True)
        torch.nn.init.xavier_uniform_(self.V.data)
        self.linear = nn.Linear(features_num, 1)

    def forward(self, X):
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(self.V[j][:, i] * self.V[i][:, j])
        ix = torch.stack(ix, dim=1)
        out_linear = self.linear(X)
        return out_interaction + out_linear

In [41]:
V = nn.Parameter(torch.randn(2, len(weights)), requires_grad=True)
torch.nn.init.xavier_uniform_(V.data)

In [49]:
V

Parameter containing:
tensor([[ 0.0032, -0.0013, -0.0022,  ..., -0.0044,  0.0084,  0.0069],
        [-0.0063, -0.0043,  0.0058,  ...,  0.0102,  0.0042, -0.0086]],
       requires_grad=True)

56036

In [13]:
weights[0]

('C1_05db9164', 'C2')

In [15]:
from itertools import combinations

In [17]:
count = 0
for _ in combinations(weights, 2):
    count +=1
    if count % 10000000 == 0:
        print(count)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
170000000
180000000
190000000
200000000
210000000
220000000
230000000
240000000
250000000
260000000
270000000
280000000
290000000
300000000
310000000
320000000
330000000
340000000
350000000
360000000
370000000
380000000
390000000
400000000
410000000
420000000
430000000
440000000
450000000
460000000
470000000
480000000
490000000
500000000
510000000
520000000
530000000
540000000
550000000
560000000
570000000
580000000
590000000
600000000
610000000
620000000
630000000
640000000
650000000
660000000
670000000
680000000
690000000
700000000
710000000
720000000
730000000
740000000
750000000
760000000
770000000
780000000
790000000
800000000
810000000
820000000
830000000
840000000
850000000
860000000
870000000
880000000
890000000
900000000
910000000
920000000
930000000
940000000
950000000
960000000
970000000
980000000
990000000
1000000000
10100000

In [18]:
count

1569988630

In [50]:
for field in features_by_field[:2]:
    for category in field["feature_names"]:
        for other_field in filter(
            lambda x: x["field_name"] != field["field_name"],
            features_by_field
        ):
            print(f"w1 {field}_{category}, other_field * ")

In [49]:
features_by_field

[{'field_name': 'C1',
  'type': 'sparse',
  'feature_names': ['05db9164',
   '09ca0b81',
   '0e78bd46',
   '17f69355',
   '241546e0',
   '2d4ea12b',
   '39af2607',
   '3b65d647',
   '52f1e825',
   '5a9ed9b0',
   '5bfa8ab5',
   '68fd1e64',
   '75ac2fe6',
   '7e5c2ff4',
   '87552397',
   '87773c45',
   '8cf07265',
   '98237733',
   '9a89b36c',
   'ae82ea21',
   'be30ca83',
   'be589b51',
   'da4eff0f',
   'de4dac42',
   'f434fac1',
   'f473b8dc',
   'fc9c62bb'],
  'feature_indeces': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26]},
 {'field_name': 'C2',
  'type': 'sparse',
  'feature_names': ['0468d672',
   '04e09220',
   '06174070',
   '064c8f31',
   '08d6d899',
   '09e68b86',
   '0a519c5c',
   '0aadb108',
   '0b8e9caf',
   '0c0567c2',
   '1cfdf714',
   '207b2d81',
   '26ece8a8',
   '287130e0',
   '2a69d406',
   '2ae0a573',
   '2c8c5f5d',
   '2eb7b10e',
   '38

In [29]:
DEFAULT_RATING_COL = "rating"

class LibffmConverter:
    """Converts an input dataframe to another dataframe in libffm format. A text file of the converted
    Dataframe is optionally generated.
    .. note::
        The input dataframe is expected to represent the feature data in the following schema:
        .. code-block:: python
            |field-1|field-2|...|field-n|rating|
            |feature-1-1|feature-2-1|...|feature-n-1|1|
            |feature-1-2|feature-2-2|...|feature-n-2|0|
            ...
            |feature-1-i|feature-2-j|...|feature-n-k|0|
        Where
        1. each `field-*` is the column name of the dataframe (column of label/rating is excluded), and
        2. `feature-*-*` can be either a string or a numerical value, representing the categorical variable or
        actual numerical variable of the feature value in the field, respectively.
        3. If there are ordinal variables represented in int types, users should make sure these columns
        are properly converted to string type.
        The above data will be converted to the libffm format by following the convention as explained in
        `this paper <https://www.csie.ntu.edu.tw/~r01922136/slides/ffm.pdf>`_.
        i.e. `<field_index>:<field_feature_index>:1` or `<field_index>:<field_index>:<field_feature_value>`, depending on
        the data type of the features in the original dataframe.
    Args:
        filepath (str): path to save the converted data.
    Attributes:
        field_count (int): count of field in the libffm format data
        feature_count (int): count of feature in the libffm format data
        filepath (str or None): file path where the output is stored - it can be None or a string
    Examples:
        >>> import pandas as pd
        >>> df_feature = pd.DataFrame({
                'rating': [1, 0, 0, 1, 1],
                'field1': ['xxx1', 'xxx2', 'xxx4', 'xxx4', 'xxx4'],
                'field2': [3, 4, 5, 6, 7],
                'field3': [1.0, 2.0, 3.0, 4.0, 5.0],
                'field4': ['1', '2', '3', '4', '5']
            })
        >>> converter = LibffmConveter().fit(df_feature, col_rating='rating')
        >>> df_out = converter.transform(df_feature)
        >>> df_out
            rating field1 field2   field3 field4
        0       1  1:1:1  2:4:3  3:5:1.0  4:4:1
        1       0  1:2:1  2:4:4  3:5:2.0  4:5:1
        2       0  1:3:1  2:4:5  3:5:3.0  4:6:1
        3       1  1:3:1  2:4:6  3:5:4.0  4:7:1
        4       1  1:3:1  2:4:7  3:5:5.0  4:8:1
    """

    def __init__(self, filepath=None):
        self.filepath = filepath
        self.col_rating = None
        self.field_names = None
        self.field_count = None
        self.feature_count = None

    def fit(self, df, col_rating=DEFAULT_RATING_COL):
        """Fit the dataframe for libffm format.
        This method does nothing but check the validity of the input columns
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            obj: the instance of the converter
        """

        # Check column types.
        types = df.dtypes
        if not all(
            [
                x == object or np.issubdtype(x, np.integer) or x == np.float
                for x in types
            ]
        ):
            raise TypeError("Input columns should be only object and/or numeric types.")

        if col_rating not in df.columns:
            raise TypeError(
                "Column of {} is not in input dataframe columns".format(col_rating)
            )

        self.col_rating = col_rating
        self.field_names = list(df.drop(col_rating, axis=1).columns)

        return self

    def transform(self, df):
        """Tranform an input dataset with the same schema (column names and dtypes) to libffm format 
        by using the fitted converter.
        Args: 
            df (pd.DataFrame): input Pandas dataframe.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        if self.col_rating not in df.columns:
            raise ValueError(
                "Input dataset does not contain the label column {} in the fitting dataset".format(
                    self.col_rating
                )
            )

        if not all([x in df.columns for x in self.field_names]):
            raise ValueError(
                "Not all columns in the input dataset appear in the fitting dataset"
            )

        # Encode field-feature.
        idx = 1
        self.field_feature_dict = {}
        for field in self.field_names:
            for feature in df[field].values:
                # Check whether (field, feature) tuple exists in the dict or not.
                # If not, put them into the key-values of the dict and count the index.
                if (field, feature) not in self.field_feature_dict:
                    self.field_feature_dict[(field, feature)] = idx
                    if df[field].dtype == object:
                        idx += 1
            if df[field].dtype != object:
                idx += 1

        self.field_count = len(self.field_names)
        self.feature_count = idx - 1

        def _convert(field, feature, field_index, field_feature_index_dict):
            field_feature_index = field_feature_index_dict[(field, feature)]
            if isinstance(feature, str):                
                feature = 1
            return "{}:{}:{}".format(field_index, field_feature_index, feature)

        for col_index, col in enumerate(self.field_names):
            df[col] = df[col].apply(
                lambda x: _convert(col, x, col_index + 1, self.field_feature_dict)
            )

        # Move rating column to the first.
        column_names = self.field_names[:]
        column_names.insert(0, self.col_rating)
        df = df[column_names]

        if self.filepath is not None:
            np.savetxt(self.filepath, df.values, delimiter=" ", fmt="%s")

        return df

    def fit_transform(self, df, col_rating=DEFAULT_RATING_COL):
        """Do fit and transform in a row
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        return self.fit(df, col_rating=col_rating).transform(df)

    def get_params(self):
        """Get parameters (attributes) of the libffm converter
        Return:
            dict: parameters field count, feature count, and file path.
        """
        return {
            "field count": self.field_count,
            "feature count": self.feature_count,
            "file path": self.filepath,
        }
    
df_feature_original = pd.DataFrame({
    'rating': [1, 0, 0, 1, 1],
    'field1': ['xxx1', 'xxx2', 'xxx4', 'xxx4', 'xxx4'],
    'field2': [3, 4, 5, 6, 7],
    'field3': [1.0, 2.0, 3.0, 2.0, 5.0],
    'field4': ['1', '2', '3', '3', '5']
})
df_feature_original

,rating,field1,field2,field3,field4
0,1,xxx1,3,1.0,1
1,0,xxx2,4,2.0,2
2,0,xxx4,5,3.0,3
3,1,xxx4,6,2.0,3
4,1,xxx4,7,5.0,5


In [21]:
converter = LibffmConverter().fit(df_feature_original, col_rating='rating')
df_out = converter.transform(df_feature_original)
df_out

,rating,field1,field2,field3,field4
0,1,1:1:1,2:4:3,3:5:1.0,4:6:1
1,0,1:2:1,2:4:4,3:5:2.0,4:7:1
2,0,1:3:1,2:4:5,3:5:3.0,4:8:1
3,1,1:3:1,2:4:6,3:5:2.0,4:8:1
4,1,1:3:1,2:4:7,3:5:5.0,4:9:1


In [10]:
d = []
for i in features_by_field:
    for j in features_by_field:
        if j["field_name"] != i["field_name"]:
            for jf in j["feature_names"]:
                if (i["type"] == "sparse") and (j["type"] == "sparse"):
                    d.append((f'{i["field_name"]}_{f}', f'{j["field_name"]}_{f}'))
#                 if j["type"] == "sparse":
#                     d.append((i["field_name"], f'{j["field_name"]}_{f}'))

len(d)

87058

$w_{ESPN,A} · w_{Nike,P} + w_{ESPN,G} · w_{Male,P} + w_{Nike,G} · w_{Male,A}$

In [12]:
list(filter(lambda x: (x[0] == "C2") and (x[1].startswith("C20")), d))

[('C2', 'C20_5840adea'),
 ('C2', 'C20_a458ea53'),
 ('C2', 'C20_b1252a9d'),
 ('C2', 'C20_missing')]

In [49]:
random.sample(d, 10)

[('C12', 'C2_404660bb'),
 ('C16', 'C3_99815367'),
 ('C21', 'C24_3b183c5c'),
 ('C14', 'C16_b6d021e8'),
 ('I10', 'C3_2273663d'),
 ('C1', 'C24_0ea7be91'),
 ('C23', 'C15_7ba31d46'),
 ('I9', 'C11_e51ddf94'),
 ('I13', 'C7_d55d70ca'),
 ('I7', 'C7_4157815a')]

In [20]:
k = 3

In [19]:
field_param = nn.Parameter(torch.empty(len(field["feature_names"]), k), requires_grad=True)

In [21]:
dp = {}
for i in features_by_field:
    for j in features_by_field:
        if j["field_name"] != i["field_name"]:
            for f in j["feature_names"]:
                if j["type"] == "dense":
                    dp[f'{i["field_name"]} * {j["field_name"]}'] = nn.Parameter(
                        torch.empty(len(i["feature_names"]), k),
                        requires_grad=True
                    )
                if j["type"] == "sparse":
                    dp[f'{i["field_name"]} * {j["field_name"]}_{f}'] = nn.Parameter(
                        torch.empty(len(i["feature_names"]), k),
                        requires_grad=True
                    )

In [ ]:
ix = []
for i in range(self.num_fields - 1):
    for j in range(i + 1, self.num_fields):
        ix.append(xs[j][:, i] * xs[i][:, j])
ix = torch.stack(ix, dim=1)

In [42]:
X_train_tensor, y_train_tensor = torch.from_numpy(criteo_dummy.values), torch.from_numpy(criteo.label.values.reshape(-1, 1))

In [ ]:
class FieldAwareFactorizationMachine(torch.nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(xs[j][:, i] * xs[i][:, j])
        ix = torch.stack(ix, dim=1)
        return ix

In [91]:
model = FM(X_train_tensor.shape[1], k=3)
criterion = nn.BCELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.001)
X_test_tensor = X_train_tensor

for i in range(5):
    for epoch in range(500):
        optimizer.zero_grad()
        predictions = model(X_train_tensor.float())
        loss = criterion(predictions, y_train_tensor.float())
        # get gradients
        loss.backward()
        # update parameters
        optimizer.step()


    predictions = model(X_test_tensor.float())
    print(roc_auc_score(criteo.label, predictions.squeeze().detach().numpy()))

0.5762265170969049
0.5488579537775375
0.5507501013650493
0.5484524935802136
0.5484524935802136


In [133]:
features_by_field

{'I1': {'type': 'dense', 'values': 'I1', 'indeces': 0},
 'I2': {'type': 'dense', 'values': 'I2', 'indeces': 1},
 'I3': {'type': 'dense', 'values': 'I3', 'indeces': 2},
 'I4': {'type': 'dense', 'values': 'I4', 'indeces': 3},
 'I5': {'type': 'dense', 'values': 'I5', 'indeces': 4},
 'I6': {'type': 'dense', 'values': 'I6', 'indeces': 5},
 'I7': {'type': 'dense', 'values': 'I7', 'indeces': 6},
 'I8': {'type': 'dense', 'values': 'I8', 'indeces': 7},
 'I9': {'type': 'dense', 'values': 'I9', 'indeces': 8},
 'I10': {'type': 'dense', 'values': 'I10', 'indeces': 9},
 'I11': {'type': 'dense', 'values': 'I11', 'indeces': 10},
 'I12': {'type': 'dense', 'values': 'I12', 'indeces': 11},
 'I13': {'type': 'dense', 'values': 'I13', 'indeces': 12},
 'C1': {'type': 'sparse',
  'values': ['C1_05db9164',
   'C1_09ca0b81',
   'C1_0e78bd46',
   'C1_17f69355',
   'C1_241546e0',
   'C1_2d4ea12b',
   'C1_39af2607',
   'C1_3b65d647',
   'C1_52f1e825',
   'C1_5a9ed9b0',
   'C1_5bfa8ab5',
   'C1_68fd1e64',
   'C1_75

In [6]:
class FFM(nn.Module):
    def __init__(self, fields: dict, embed_dim=3):
        super().__init__()
        self.fields = fields
    
        for field in fields:
            field["embedding"] = nn.Parameter(torch.randn(len(field["feature_names"]), k), requires_grad=True)
            torch.nn.init.xavier_uniform_(self.V.data)
            
        torch.nn.init.xavier_uniform_(self.V.data)
        self.linear = nn.Linear(features_num, 1)

    def forward(self, X):
        out_1 = ((X @ self.V) ** 2).sum(1, keepdim=True)
        out_2 = ((X ** 2) @ (self.V ** 2)).sum(1, keepdim=True)

        out_interaction = (out_1 - out_2) / 2
        out_linear = self.linear(X)
        return out_interaction + out_linear

In [4]:
from torch import optim

X_train_tensor, y_train_tensor = torch.from_numpy(X_train.toarray()), torch.from_numpy(y_train.reshape(-1, 1))

model = FM(X_train.shape[1])
criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(500):
    optimizer.zero_grad()
    predictions = model(X_train_tensor.float())
    loss = criterion(predictions, y_train_tensor.float())
    # get gradients
    loss.backward()
    # update parameters
    optimizer.step()
    
X_test_tensor = torch.from_numpy(X_test.toarray())
predictions = model(X_test_tensor.float())
r2_score(y_test, predictions.squeeze().detach().numpy())

0.9788741139133923

In [ ]:
1

http://ailab.criteo.com/ctr-prediction-linear-model-field-aware-factorization-machines/